In [59]:
import numpy as np
import math

In [55]:
def read_file(fileName):

    POS_TRACK_ID = 2
    POS_X = 4
    POS_Y = 5
    POS_T = 7

    X = []
    Y = []
    T = []
    TRACK_ID = []
    with open(fileName,'r') as file:
        lines = file.read().split("\n")
        terminos = lines[1].split(",") #con lines[1] me dirijo a la primera linea de datos
        trackId = int(terminos[POS_TRACK_ID]) #obtengo el primer trackId
        x = []
        y = []
        t = []
        i = 1
        while True:
            #print('hola')
            if lines[i]:
                terminos = lines[i].split(",")
                if trackId != int(terminos[POS_TRACK_ID]): #tengo un nuevo trackId
                    TRACK_ID.append(trackId)
                    trackId = int(terminos[POS_TRACK_ID])
                    #print(trackId)
                    X.append(x)
                    Y.append(y)
                    T.append(t)
                    x = []
                    y = []
                    t = []
                elif t == [] or float(terminos[POS_T]) != t[-1]:
                    x.append(float(terminos[POS_X]))
                    y.append(float(terminos[POS_Y]))
                    t.append(float(terminos[POS_T]))
                    i = i+1
                else:
                    i = i + 1
            else:
                X.append(x)
                Y.append(y)
                T.append(t)
                TRACK_ID.append(trackId)
                break

    file.close()
    return X,Y,T,TRACK_ID

In [225]:
filename = 'Spots in tracks statistics.csv'
prueba_1track = 'prueba_1track.csv'
#X,Y,T,TRACK_ID = read_file(filename)
X,Y,T,TRACK_ID = read_file(prueba_1track)

[0, 2, 0, 91]

In [219]:
def avgPath(X,Y):
    
    NumberNewPoints = 5
    xPath = []
    yPath = []
    
    for i in range(len(X)-1):    
        xvals = np.linspace(X[i], X[i+1], NumberNewPoints+2) #crear 5 puntos en el medio (7 en total)
        yvals = np.linspace(Y[i], Y[i+1], NumberNewPoints+2)
        if i != 0:
            xvals = xvals[1:-1] #saco el primer y ultimo lugar
            yvals = yvals[1:-1]
        for j in range(len(xvals)):
            xPath.append(xvals[j]) #obtengo caminos con muchos puntos intermedios (5 entre cada dos puntos originales)    
            yPath.append(yvals[j])
    
    
    windowSize = NumberNewPoints*3     
    xPathSmooth = [] #se promedia con una ventana - al final se obtiene un camino con #windowSize puntos menos
    yPathSmooth = [] #la ventana movil se hace desde 0+windowsize/2 hasta fin-windowsize/2
    for i in range(math.trunc(windowSize/2),int(len(xPath)-math.ceil(windowSize/2))):
        xPathSmooth.append( np.mean (xPath [int(i-math.trunc(windowSize/2)):int(i+math.ceil(windowSize/2))] ) )
        yPathSmooth.append( np.mean (yPath [int(i-math.trunc(windowSize/2)):int(i+math.ceil(windowSize/2))] ) )    
    
    #uno el primer y ultimo punto con el avg path asi me queda una avg path completo

    xvals1 = np.linspace(X[0], xPathSmooth[0], math.ceil(windowSize/2)+2) #se completa la parte del smoothpath que
    yvals1 = np.linspace(Y[0], yPathSmooth[0], math.ceil(windowSize/2)+2) #falta entre el primer valor y el primero
    xvalsEnd = np.linspace(xPathSmooth[-1],X[-1], math.ceil(windowSize/2)+2) #del array smooth
    yvalsEnd = np.linspace(yPathSmooth[-1],Y[-1], math.ceil(windowSize/2)+2)    
    for i in range(len(xvals1)-1):
        xPathSmooth = [xvals1[-i-2]]+xPathSmooth+[xvalsEnd[i+1]]
        yPathSmooth = [yvals1[-i-2]]+yPathSmooth+[yvalsEnd[i+1]]

    return xPathSmooth, yPathSmooth

def VSL(X,Y,T):
    dist = math.sqrt((X[-1]-X[0])**2+(Y[-1]-Y[0])**2)
    time = T[-1]-T[0]
    vsl = dist/time
    return vsl

def VCL(X,Y,T):
    vel = []
    for i in range(len(X)-1):
        dist = math.sqrt((X[i+1]-X[i])**2+(Y[i+1]-Y[i])**2)
        time = T[i+1]-T[i]
        vel.append(dist/time)
    vcl = np.mean(vel)
    return vcl

def VAP(X,Y,avgPathX,avgPathY,T):
    vel = []
    minIndexOld = 0        
    for j in range(1,len(X)):
        minDist = float('Inf')
        for i in range (len(avgPathX)):
            dist = math.sqrt((X[j]-avgPathX[i])**2+(Y[j]-avgPathY[i])**2)
            if dist < minDist:
                minIndex = i
                minDist = dist
        dist = 0
        if minIndex>=minIndexOld:
            for i in range(minIndexOld,minIndex):
                dist = dist + math.sqrt((avgPathX[i+1]-avgPathX[i])**2+(avgPathY[i+1]-avgPathY[i])**2) 
        else:
            for i in range(minIndex,minIndexOld):
                dist = dist - math.sqrt((avgPathX[i+1]-avgPathX[i])**2+(avgPathY[i+1]-avgPathY[i])**2)         
        minIndexOld = minIndex
        time = T[j]-T[j-1]
        vel.append(dist/time)
    vap = np.mean(vel)
    return vap

def ALH(X,Y,avgPathX,avgPathY):
    alh = []        
    for j in range(len(X)):
        minDist = float('Inf')
        for i in range (len(avgPathX)):
            dist = math.sqrt((X[j]-avgPathX[i])**2+(Y[j]-avgPathY[i])**2)
            if dist < minDist:
                minDist = dist
        alh.append(minDist)
    return np.mean(alh)

def LIN(X,Y,T):
    lin = VSL(X,Y,T)/VCL(X,Y,T)
    return lin

def WOB(X,Y,avgPathX,avgPathY,T):
    wob = VAP(X,Y,avgPathX,avgPathY,T)/VCL(X,Y,T)
    return wob

def STR(X,Y,avgPathX,avgPathY,T):
    stra = VSL(X,Y,T)/VAP(X,Y,avgPathX,avgPathY,T)
    return stra

def BCF(X,Y,avgPathX,avgPathY,T):
    bcf = []
    for j in range(1,len(X)):
        minDist = float('Inf')
        for i in range (len(avgPathX)):
            dist = math.sqrt((X[j]-avgPathX[i])**2+(Y[j]-avgPathY[i])**2)
            if dist < minDist:
                minIndexNew = i
                minDist = dist
        if j>1:
            Ax = avgPathX[minIndexOld]
            Ay = avgPathY[minIndexOld]
            Bx = avgPathX[minIndexNew]
            By = avgPathY[minIndexNew]
            discNew = (Bx - Ax) * (Y[j] - Ay) - (By - Ay) * (X[j] - Ax)
            discOld = (Bx - Ax) * (Y[j-1] - Ay) - (By - Ay) * (X[j-1] - Ax)
            if discOld*discNew<0:
                bcf.append(1)
            else:
                bcf.append(0)
        minIndexOld = minIndexNew
    return np.mean(bcf)

def MAD(X,Y):
    mad = []
    for i in range(1,len(X)-1):
        if (X[i]-X[i-1])!= 0:
            pend1 = (Y[i]-Y[i-1])/(X[i]-X[i-1])
            angle1 = math.atan(pend1)
            if (pend1 < 0 and X[i]<X[i-1]) or (pend1 > 0 and X[i]<X[i-1]):
                angle1 = angle1+math.pi
            else:
                angle1 = 2*math.pi+angle1
        elif Y[i]>Y[i-1]:
            angle1 = math.pi/2
        else: 
            angle1 = -math.pi/2
        if (X[i+1]-X[i])!= 0:
            pend2 = (Y[i+1]-Y[i])/(X[i+1]-X[i])            
            angle2 = math.atan(pend2)
            if (pend2 < 0 and X[i+1]<X[i]) or (pend2 > 0 and X[i+1]<X[i]):
                angle2 = angle2+ math.pi
            else:
                angle2 = 2*math.pi+angle2
        elif Y[i+1]>Y[i]:
            angle2 = math.pi/2
        else: 
            angle2 = -math.pi/2
        mad.append(abs(angle1-angle2))
    return np.mean(mad)

In [223]:
def get_carac(X,Y,T,min_detections):

    allX = X
    allY = Y
    allT = T
    
    CARAC_WHO = []
    CARAC_SIX = []

    
    
    for i in range(len(allX)):
        
        X = allX[i]
        Y = allY[i]
        T = allT[i]
        
        if ((np.shape(X)[0]) > min_detections):  
            avgPathX, avgPathY = avgPath(X,Y)
            vcl = VCL(X,Y,T)
            vsl = VSL(X,Y,T)
            vap = VAP(X,Y,avgPathX,avgPathY,T)
            alh = ALH(X,Y,avgPathX,avgPathY)
            lin = LIN(X,Y,T)
            wob = WOB(X,Y,avgPathX,avgPathY,T)
            stra = STR(X,Y,avgPathX,avgPathY,T)
            bcf = BCF(X,Y,avgPathX,avgPathY,T)
            mad = MAD(X,Y)
            
        else:
            vcl = 0
            vsl = 0
            vap = 0
            alh = 0
            lin = 0
            wob = 0
            stra = 0
            bcf = 0
            mad = 0
            
        
        carac_six = [vcl,vsl,vap,lin,bcf,mad]
        CARAC_SIX.append(carac_six)
        
        
        carac_who = [vcl,vsl,vap,alh,lin,wob,stra,bcf,mad]
        CARAC_WHO.append(carac_who)


    return CARAC_SIX, CARAC_WHO

In [224]:
X,Y,T,TRACK_ID = read_file(filename)
carac6, carac_who = get_carac(X,Y,T,min_detections=11)
print(carac6)

[[0.5981222812667668, 0.34229173286173226, 0.3696218423276061, 0.5722771807410192, 0.11392405063291139, 2.0957277373888457], [0.3892806263524995, 0.025286117833745257, 0.22109679797536227, 0.06495601404742987, 0.4745762711864407, 2.13801942139562], [1.2000837619557818, 0.8921001785292144, 1.010008230180465, 0.7433649273575328, 0.29411764705882354, 0.6623848311701926], [0.35065292639659995, 0.0076303492756556, 0.1742898626763784, 0.02176040381030622, 0.4576271186440678, 2.2787585710159757], [1.186019325048177, 0.6794994482648491, 0.7734399901129385, 0.5729244321016838, 0.42, 1.7694086976365222], [2.7778528737391572, 0.8748298788552377, 1.5187866329735347, 0.3149302423917304, 0.37254901960784315, 1.3775624359856846], [2.458415789031803, 1.0250907720866735, 1.6324604786804804, 0.4169720909945769, 0.475, 2.4505852613184267], [2.0368610034543004, 0.6317007759597414, 1.2851907762988073, 0.31013445438272114, 0.4067796610169492, 1.8796236512195947], [2.0471127157458717, 1.053307687357053, 1.34